## Description

Description:

Filter the three main data sources (Fund Info, Returns and Holdings)
based on some parameters and save the result.

Approach:
1. Match fund summary to each portfolio/date pair
2. Filter fund/date pairs based on those infos
3. Filter returns based on final sample of holdings

Parameters: 
- Obj_codes
- Percentage
- Flags

In [1]:
%matplotlib inline

import os
import sys

# add the 'src' directory as one where we can import modules
src_dir = os.path.join(os.getcwd(), os.pardir, 'src')
sys.path.append(src_dir)
from data.basic_functions import * 

import numpy as np
import pandas as pd

from scipy import sparse

import matplotlib.pyplot as plt

import datetime

# For multiprocessing
import multiprocessing
from itertools import product

### Load the data files

In [2]:
data_path = '../data/raw/'

returns = load_data(data_path,'monthly_returns')
summary = load_data(data_path,'total_summary_new') # Wo kommt das her ???
summary.shape

(167176, 8)

In [3]:
npz_path = '../data/interim/sparse_matrix_t.npz'
holdings = sparse.load_npz(npz_path)
holdings.shape

(733743, 2382968)

In [4]:
data_path = '../data/interim/'
holdings_summary = load_data(data_path,'sparse_info_t')
holdings_summary.shape

(733743, 2)

In [5]:
path = '../data/raw/portno_map.feather'
portno_map = feather.read_dataframe(path)

## Drop duplicates in summary data

To increase speed of matching obj_codes and other fund info to portfolios

In [6]:
print('Shape of summary before cleaning is: {:,} / {:,}'.format(summary.shape[0],summary.shape[1]))

Shape of summary before cleaning is: 167,176 / 8


In [7]:
# Delet not nas
summary_clean = summary[summary['crsp_portno'].notna()]

# Drop duplicates based on all but two columns
summary_clean = summary_clean.drop_duplicates(summary_clean.columns.difference(['crsp_fundno','fund_name']))

summary_clean.loc[:,'crsp_fundno'] = pd.to_numeric(summary_clean.loc[:,'crsp_fundno'], downcast='integer')
summary_clean.loc[:,'crsp_portno'] = pd.to_numeric(summary_clean.loc[:,'crsp_portno'], downcast='integer')

In [8]:
print('Shape of summary data after cleaning: {:,} / {:,}'.format(summary_clean.shape[0],summary_clean.shape[1]))

Shape of summary data after cleaning: 96,077 / 8


# Match obj code to portfolios

TODO: 

Fund info flags like the index_fund_flag could in theory also be different in the fund_history

Therefore a similar approach should also be used for those items

In general portno fundno map also beginning and end times

In [44]:
def port_ID_to_port_info(fund_info):  
            """
            Used to merge the right obj_code and other fund info 
            to each holdings_info row and therefore to each row of the sparse holdings matrix
            
            Input:
            - fund_info: Tuple consisting of the port_no (1st element) and the report_dt (2nd)
            
            Output:
            - Tuple of port_no, report_dt, index_fund_flag, et_flag and crsp_obj_cd. 
                NaN if value is not available
            
            Attention:
            Depends on global variable summary to look up the values
            Must be renamed or changed in the function
            """
            port_no = fund_info[0]
            report_dt = fund_info[1].date() # Implement in large csv file
            mask = summary['crsp_portno'].values == port_no
            my_class = summary.loc[mask]

            my_class_n = my_class.loc[
                (my_class.begdt <= report_dt) & 
                (my_class.enddt >= report_dt)]
            
            try:
                crsp_obj_cd = my_class_n['crsp_obj_cd'].values[0]
                index_fund_flag = my_class_n['index_fund_flag'].values[0]
                et_flag = my_class_n['et_flag'].values[0]

            except:
                crsp_obj_cd = np.nan
                index_fund_flag = np.nan
                et_flag = np.nan
                
            return(port_no, report_dt, index_fund_flag, et_flag, crsp_obj_cd)        

### Multiprocessing

In [39]:
a = holdings_summary['port_no']
b = holdings_summary['date']

fund_info = list(zip(a,b))

In [57]:
len(fund_info)

733743

In [55]:
%%time
with multiprocessing.Pool(processes=8) as pool:
    results = pool.map(port_ID_to_port_info, fund_info)

CPU times: user 369 ms, sys: 100 ms, total: 469 ms
Wall time: 54.5 s


In [58]:
labels = ['port_no','report_dt','index_fund_flag','et_flag','crsp_obj_cd']
holdings_summary = pd.DataFrame.from_records(results, columns=labels)

holdings_summary.shape

(100000, 5)

In [59]:
holdings_summary[holdings_summary['crsp_obj_cd'].isna()].shape # -> For some portfolios there is simly no row in fund_header

(83466, 5)

### Out of the roughly 730k portfolios, for 129k there is no fund header info available

In [65]:
holdings_summary.sample(10)

,port_no,report_dt,index_fund_flag,et_flag,crsp_obj_cd
51130,1001777,2008-09-30,NaN,NaN,NaN
81822,1003326,2006-10-31,None,None,EDCS
38782,1001325,2005-09-30,NaN,NaN,NaN
12307,1000399,2008-10-31,NaN,NaN,NaN
31330,1001054,2010-02-27,NaN,NaN,NaN
1722,1000052,2007-12-31,None,None,EDYB
99648,1004670,2006-11-30,NaN,NaN,NaN
12683,1000413,2009-08-31,NaN,NaN,NaN
78461,1003043,2007-06-30,NaN,NaN,NaN
19828,1000663,2009-04-30,NaN,NaN,NaN


# Clean holdings_summary data

Do not delet rows since they match to the sparse matrix

TODO: Replace all falgs with proper categories

In [66]:
# Make the two flags categories and rename those categories accordingly
holdings_summary[['et_flag','index_fund_flag']] = holdings_summary[['et_flag','index_fund_flag']].astype('category')

et_mapper = {'F':'ETF', 'N':'ETN', np.nan:'MF'}
holdings_summary['et_flag'] = holdings_summary['et_flag'].map(et_mapper)

index_flag_mapper = {'B':'Index-based', 'D':'Pure Index', 'E':'Index enhanced', np.nan:'MF'}
holdings_summary['index_fund_flag'] = holdings_summary['index_fund_flag'].map(index_flag_mapper)

In [67]:
# Creat new flag var 'mutual_fund' that is Y for Mutual Funds and N for other funds
holdings_summary.loc[(holdings_summary['index_fund_flag'] == 'MF') & 
                     (holdings_summary['et_flag'] == 'MF'),'mutual_fund'] = 'Y'
holdings_summary.loc[holdings_summary['mutual_fund'].isna(),'mutual_fund'] = 'N'

In [68]:
# Creat new flag var 'sample' that is Y for those included and N for those not included
# , 'EDCL', 'EDCM', 'EDCS', 'EDCI'
# selected_obj_codes = ('EDYG','EDYB' ,'EDYH' ,'EDYS' ,'EDYI')

selected_obj_codes = ('EDYG','I')

holdings_summary.loc[(holdings_summary['mutual_fund'] == 'Y') & 
                     (holdings_summary['crsp_obj_cd'].isin(selected_obj_codes)),'sample'] = 'Y'
holdings_summary.loc[holdings_summary['sample'].isna(),'sample'] = 'N'

In [69]:
# Make the two new variables categorical
holdings_summary[['mutual_fund','sample']] = holdings_summary[['mutual_fund','sample']].astype('category')

In [70]:
holdings_summary['sample'].value_counts()

N    95886
Y     4114
Name: sample, dtype: int64

In [71]:
pd.crosstab(holdings_summary['et_flag'],holdings_summary['index_fund_flag'])

index_fund_flag,Index enhanced,Index-based,MF,Pure Index
et_flag,,,,
ETF,0,0,3,64
MF,94,64,99105,670


In [72]:
pd.crosstab(holdings_summary['mutual_fund'],holdings_summary['sample'])

sample,N,Y
mutual_fund,,
N,895,0
Y,94991,4114


#### 148k Funds in EDY
#### 202k Funds in EDC

# Save holdings_summary

In [ ]:
path = '../data/interim/holdings_summary_total.feather'
#feather.write_dataframe(holdings_summary,path)

In [ ]:
path = '../data/interim/holdings_summary_total.feather'
holdings_summary = feather.read_dataframe(path)

In [ ]:
holdings.shape

In [ ]:
holdings_summary.shape

### Add fund_no to holdings_summary

#### Fundo is not an integer for now but not that important -> TODO

In [ ]:
portno_map_unique = portno_map.drop_duplicates(subset='crsp_portno')

#### Maybe must be modified since all but one associated fund_nos per portfolio are deleted 

In [ ]:
holdings_summary = holdings_summary.merge(portno_map_unique[['crsp_portno','crsp_fundno']],how='left', left_on = 'port_no', right_on='crsp_portno')

In [ ]:
holdings_summary.shape

In [ ]:
mask = holdings_summary['crsp_fundno'].notna()
holdings_summary['crsp_fundno'] = holdings_summary.loc[mask,'crsp_fundno'].astype(int)

In [ ]:
new_order = [0,8,1,2,3,4,5,6]
holdings_summary = holdings_summary[holdings_summary.columns[new_order]]

In [ ]:
holdings_summary = holdings_summary.rename(columns={'crsp_fundno':'fund_no'}, index=str)

# Take sample according to parameter

### Filter returns

In [ ]:
unique_portno = holdings_summary[['fund_no']].drop_duplicates()

In [ ]:
mask = returns['crsp_fundno'].isin(unique_portno['fund_no'])

In [ ]:
returns_s = returns[mask]

In [ ]:
returns.shape

In [ ]:
returns_s.shape

#### From 7.2m to 2.1m return datapoints

# Filter holdings

### Mask to filter out only those in the sample according to holdings_summary

In [ ]:
mask = (holdings_summary['sample'] == 'Y') 
np.sum(mask)

In [ ]:
mask = (holdings_summary['sample'] == 'Y') & (holdings_summary['port_no'].notna())
np.sum(mask)

In [ ]:
holdings_s = holdings[mask.values]
holdings_s

### Filter holdings summary

In [ ]:
holdings_summary_s = holdings_summary[mask]

In [ ]:
holdings_s.shape

In [ ]:
holdings_summary_s.shape

### Take only last n number of obs per fund

To avoid overweight of funds with many observations

In [ ]:
holdings_summary_s = holdings_summary_s.reset_index()
index = pd.DataFrame(np.arange(holdings_summary_s.shape[0]))
index = index.groupby(holdings_summary_s['port_no']).tail(10)
index = index.values.T.flatten()

In [ ]:
holdings_summary_s = holdings_summary_s.reset_index().loc[index,:]
holdings_s = holdings_s[index]
holdings_s.shape

### Delet columns with little to no information

In [ ]:
holdings_b = sparse.csr_matrix(holdings_s, copy=True)
holdings_b.data = np.ones(len(holdings_s.data))

In [ ]:
col_sums = pd.DataFrame(holdings_b.sum(0)).values.flatten()

1.5m of the 2m securities have either zero or one entry 

In [ ]:
# generate mask
mask = col_sums > 1000

In [ ]:
holdings_s = holdings_s.tocsc()
holdings_s = holdings_s[:,mask]
holdings_s = holdings_s.tocsr()
holdings_s

In [ ]:
holdings_b = holdings_b.tocsc()
holdings_b = holdings_b[:,mask]
holdings_b = holdings_b.tocsr()
holdings_b

### Save final cleaned and filtered data

#### Sparse matrix

In [ ]:
path = '../data/processed/EDY/holdings_s'
sparse.save_npz(path, holdings_s)

In [ ]:
path = '../data/processed/EDY/holdings_b'
sparse.save_npz(path, holdings_b)

#### Sparse info

In [ ]:
path = '../data/processed/EDY/holdings_summary_s.feather'
feather.write_dataframe(holdings_summary_s,path)

#### Returns

In [ ]:
path = '../data/processed/EDY/returns_s.feather'
feather.write_dataframe(returns_s,path)

## Take smaller sub_sub sample (Everything before specified year)
Makes processing faster

In [ ]:
start_date = '2015-01-01'
end_date = '2018-01-01'

#### Holdings & holdings_summary

In [ ]:
mask = (holdings_summary_s['report_dt'] > start_date) & (holdings_summary_s['report_dt'] < end_date)

In [ ]:
holdings_s_s = holdings_s[mask.values]

In [ ]:
holdings_summary_s_s = holdings_summary_s[mask]

#### Test

In [ ]:
holdings_s_s.shape

In [ ]:
holdings_summary_s_s.shape

#### Returns

In [ ]:
mask = (returns_s['caldt'] > start_date) & (returns_s['caldt'] < end_date)
returns_s_s = returns_s[mask]

### Save final cleaned and filtered data

#### Sparse matrix

In [ ]:
path = '../data/processed/EDY/holdings_s_s'
sparse.save_npz(path, holdings_s_s)

#### Sparse info

In [ ]:
path = '../data/processed/EDY/holdings_summary_s_s.feather'
feather.write_dataframe(holdings_summary_s_s,path)

#### Returns

In [ ]:
path = '../data/processed/EDY/returns_s_s.feather'
feather.write_dataframe(returns_s_s,path)